In [15]:
import codecs
import io, json, codecs
import os
import fix

In [41]:
def redif_json(journal_name,redif_file):
    global articles; articles=[]
    global article; article={}
    global creators; creators=[]
    global creator; creator={}
    global e
    for i in content:
        pair=i.split(':')
        j=i.replace(pair[0]+':','').strip()
        key=pair[0].lower()
        if 'template-type' in key:
            if len(article)>0:
                if len(creator)>0:
                    creators.append(creator) 
                    creator={}
                article['ex:creator']=creators
                creators=[]
                if fix.validate(article)==True:
                    articles.append(article)
                else:
                    e=e+1
            article={}
            article['ex:creator']=[]
            article['ex:keyword']=[]
            article['ex:source']=journal_name
            article['ex:year']=''
            article['ex:title']=''
            #article['@context']={"dc": "http://schema.org/"}
            #article['ex:template']=i
        elif key=='author-name':
            if len(creator)>0:
                creators.append(creator)
            creator={}
            if ',' in j:
                j=' '.join(reversed([part.strip() for part in j.split(',')]))
            creator['ex:name']=j
        elif key =='author-email':
            creator['ex:email']=j
        elif key=='author-workplace-Name':
            creator['ex:affiliation']=j
        elif key=='title':article['ex:title']=j
        elif key=='year':article['ex:date']=j
        elif key=='pages':article['ex:pages']=j
        elif key=='volume':article['ex:volume']=j
        elif key=='issue':article['ex:issue']=j
        elif key=='file-url':article['ex:url']=j
        elif key=='abstract':article['ex:abstract']=j
        elif key=='keywords':
            for keyword in j.replace(';',',').split(','):
                if keyword.strip()!='':
                    article['ex:keyword'].append(fix.fix_keyword(keyword))

    if len(article)>0:
        if len(creator)>0:
            creators.append(creator) 
            creator={}
        article['ex:creator']=creators
        creators=[]
        articles.append(article)   
    jd=unicode(json.dumps(articles, ensure_ascii=False))
    if redif_file[-4:]=='.rdf': 
        jsonld_file=jsonld_folder+redif_file.replace('.rdf','.jsonld').replace('/','_')
    if redif_file[-6:]=='.redif': 
        jsonld_file=jsonld_folder+redif_file.replace('.redif','.jsonld').replace('/','_')
    with io.open(jsonld_file, 'w') as f:
        f.write(jd)
    return (jsonld_file)

In [ ]:
with open('journals.json') as data_file:    
    journals = json.load(data_file)

redif_folder='redif/'
jsonld_folder='jsonld/'

for journal in journals:
    source=journal['jname']
    journal_name=journal['journal']
    data=journal['data']
    for type_pub in data:   
        file_type=type_pub['type']
        publisher=type_pub['publisher']
        if file_type=='redif':
            print publisher,source,journal_name,
            i=0
            e=0
            for file_name in os.listdir(redif_folder+publisher+'/'+source):
                if file_name[-4:]=='.rdf' or file_name[-6:]=='.redif':
                    redif_file=publisher+'/'+source+'/'+file_name
                    content = []
                    jd=''
                    articles=[]
                    with codecs.open(redif_folder+redif_file,encoding='utf-8',errors='ignore') as file:
                        for line in file:
                            line = line.strip()
                            content.append(line)
                    jsonld_file=redif_json(journal_name,redif_file)
                i=i+1
            print 'fixes=',e,'files=',i

eee aosoci Accounting, Organizations and Society fixes= 5 files= 1
aea aecrev American Economic Review

In [29]:
article

{'ex:creator': [{'ex:name': u'Jens Schmidt'},
  {'ex:name': u'Richard Makadok'},
  {'ex:name': u'Thomas Keil'}],
 'ex:date': u'2016',
 'ex:issue': u'9',
 'ex:keyword': [],
 'ex:pages': u'2003-2007',
 'ex:source': u'Strategic Management Journal',
 'ex:title': 'letter to the editor',
 'ex:url': u'http://hdl.handle.net/10.1002/smj.2547',
 'ex:volume': u'37',
 'ex:year': ''}

In [28]:
article['ex:title']='letter to the editor'
fix.validate(article)

False